In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self,vocab_size,hidden_size,window_size,corpus):
        V,H=vocab_size,hidden_size

        #重みの初期化
        W_in=0.01*np.random.randn(V,H).astype('f')
        W_out=0.01*np.random.randn(V,H).astype('f')

        #レイヤの生成
        self.in_layers=[]
        for i in range(2*window_size):  #window_sizeの２倍（左側と右側,=contextsの数）だけEmbeddingレイヤを用意する
            layer=Embedding(W_in)
            self.in_layers.append(layer)
        
        self.ns_loss=NegativeSamplingLoss(W_out,corpus,power=0.75,sample_size=5)

        #パラメータの格納
        self.params,self.grads=[],[]
        layers=self.in_layers+[self.ns_loss]

        for layer in layers:
            self.params+=layer.params
            self.grads+=layer.grads

        #メンバ変数に単語の分散表現を設定
        self.word_vecs=W_in

    def forward(self,contexts,target):
        h=0
        for i,layer in enumerate(self.in_layers):
            h+=layer.forward(contexts[:,i])
        h*=1/len(self.in_layers)    #出力の総和をコンテキスト数で割って平均を取る
        loss=self.ns_loss.forward(h,target)
        return loss

    def backward(self,dout=1):
        dout=self.ns_loss.backward(dout)
        dout*=1/len(self.in_layers) #乗算ノードの逆伝搬
        for layer in self.in_layers:
            layer.backward(dout)
        return None

ModuleNotFoundError: ignored